<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# Exploration et Nettoyage des données

## Librairies

In [17]:
#Importation des librairies utilisées
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
import collections
import itertools
import warnings

from sklearn.cross_validation import train_test_split

Si vous utilisez la librairie `nltk` pour la première fois, il est nécessaire d'utiliser la commande suivante. Cette commande permet de télécharger de nombreux corpus de texte, mais également des informations grammaticales sur différentes langues. Information notamment nécessaire à l'étape de racinisation.

In [18]:
# nltk.download()

## Importation des données

Le jeu de données utilisé ici est un sous ensemble du jeu de données fourni par par CDiscount et paru sur le site [datascience.net](https://www.datascience.net/fr/challenge). 
Les données d'apprentissage sont accessibles sur demande auprès de CDiscount. Les solutions de l'échantillon test du concours ne sont pas et ne seront pas rendues publiques. Un échantillon test est donc construit pour l'usage de ce tutoriel.  L'objectif est de prévoir la catégorie d'un produit à partir de son descriptif. Seule la catégorie principale (1er niveau) est prédite au lieu des trois niveaux demandés dans le concours. L'objectif est plutôt de comparer les performances des méthodes et technologies en fonction de la taille de la base d'apprentissage ainsi que d'illustrer sur un exemple complexe le prétraitement de données textuelles. La stratégie de sous ou sur échantillonnage des catégories qui permet d'améliorer la prévision n'a pas été mise en oeuvre.
* L'exemple est présenté sur un échantillon réduit d'un million de produits au lieu des 15M initiaux
* L'échantillon réduit peut encore l'être puis séparé en 2 parties: apprentissage et validation. 
* Les données textuelles sont  nettoyées, racinisées, vectorisées avant modélisation.
* Trois modélisations sont estimées: logistique, arbre, forêt aléatoire.
* Optimiser l'erreur en faisant varier différents paramètres: types et paramètres de vectorisation (TF-IDF), paramètres de la régression logistique (pénalisation l1) et de la forêt aléatoire (nombre d'arbres et nombre de variables aléatoire).

Exécuter finalement le code pour différentes tailles (paramètre  `tauxTot` ci-dessous) de l'échantillon d'apprentissage et comparer les qualités de prévision obtenues. 

Deux échantillons de test ont été mis de côté et seront utilisés dans un prochain calepin (avec pyspark) pour comparer les stratégies.

In [16]:
# Nom des fichiers
input_path = "data/cdiscount.csv"

data = pd.read_csv(input_path,sep=",")

   ### Read & Split Dataset
   Fonction permettant de lire le fichier d'apprentissage et de créer deux DataFrame Pandas, un pour l'apprentissage, l'autre pour la validation.
   La première méthode créée un DataFrame en lisant entièrement le fichier. Puis elle scinde le DataFrame en deux  grâce à la fonction dédiée de sklearn. 

In [14]:
def split_dataset(input_path, nb_line, tauxValid,columns):
    time_start = time.time()
    data_all = pd.read_csv(input_path,sep=",")
    data_all = data_all.fillna("")
    data_train, data_valid = train_test_split(data_all, test_size = tauxValid)
    time_end = time.time()
    print("Split Takes %d s" %(time_end-time_start))
    return data_train, data_valid

nb_line=10000000  # part totale extraite du fichier initial ici déjà réduit
data_train, data_valid = split_dataset(training_reduit_path, nb_line, tauxValid, HEADER_TRAIN)
# Cette ligne permet de visualiser les 5 premières lignes de la DataFrame 
N_train = data_train.shape[0]
N_valid = data_valid.shape[0]
print("Train set : %d elements, Test set : %d elements" %(N_train, N_valid))
data_train.head(5)

Split Takes 6 s
Train set : 900000 elements, Test set : 100000 elements


,Categorie1,Categorie2,Categorie3,Description,Libelle,Marque
146828,ELECTROMENAGER,GROS APPAREILS LAVAGE-SECHAGE,ACCESSOIRES ET PIECES - LAVAGE-SECHAGE,Ressort fixation manchette - Ressort fixation ...,Ressort fixation manchette,BRANDT
681349,DECO - LINGE - LUMINAIRE,DECORATION MURALE - TABLEAU - CADRE PHOTO - ST...,TABLEAU - TOILE,Tableau deco moderne BROOKLYN and MANHATTAN ...,Tableau deco moderne BROOKLYN and MANHATTAN ...,ELITE
317614,LIBRAIRIE,AUTRES LIVRES,AUTRES LIVRES,De Germinal aux éditions SOCIETE DES ECRIVAINS,RETRAITES ET SANTE : POIL A GRATTER,
209372,TELEPHONIE - GPS,ACCESSOIRE TELEPHONE,COQUE - BUMPER - FACADE TELEPHONE,Coque souple Noire pour LG G3 D851 motif Drape...,Coque souple Noire pour LG G3 D851 motif Drapea…,MUZZANO
15259,TELEPHONIE - GPS,ACCESSOIRE TELEPHONE,COQUE - BUMPER - FACADE TELEPHONE,Coque de protection rigide. Design original d'...,Coque Galaxy S4 Design Skull and bones raiddog,SKINKIN


In [5]:
data_valid.to_csv(DATA_DIR + "Categorie_reduit_valid.csv", index=False)
data_train.to_csv(DATA_DIR + "Categorie_reduit_train.csv", index=False)

## 2. Nettoyage des données
Afin de limiter la dimension de l'espace des variables ou *features*, tout en conservant les informations essentielles, il est nécessaire de nettoyer les données en appliquant plusieurs étapes:
* Chaque mot est écrit en minuscule.
* Les termes numériques, de ponctuation et autres symboles sont supprimés.
* 155 mots-courants, et donc non informatifs, de la langue française sont supprimés (STOPWORDS). Ex: le, la, du, alors, etc...
* Chaque mot est "racinisé", via la fonction `STEMMER.stem` de la librairie nltk. La racinisation transforme un mot en son radical ou sa racine. Par exemple, les mots: cheval, chevaux, chevalier, chevalerie, chevaucher sont tous remplacés par "cheva".

### Importation des librairies et fichier pour le nettoyage des données.

In [6]:
# Librairies 
from bs4 import BeautifulSoup #Nettoyage d'HTML
import re # Regex
import nltk # Nettoyage des données

## listes de mots à supprimer dans la description des produits
## Depuis NLTK
nltk_stopwords = nltk.corpus.stopwords.words('french') 
## Depuis Un fichier externe.
lucene_stopwords =open(DATA_DIR+"lucene_stopwords.txt","r").read().split(",") #En local
## Union des deux fichiers de stopwords 
stopwords = list(set(nltk_stopwords).union(set(lucene_stopwords)))

## Fonction de setmming de stemming permettant la racinisation
stemmer=nltk.stem.SnowballStemmer('french')

### Fonction de nettoyage de texte
Fonction qui prend en intrée un texte et retourne le texte nettoyé en appliquant successivement les étapes suivantes: Nettoyage des données HTML, conversion en texte minuscule, encodage uniforme, suppression des caractéres non alpha numérique (ponctuations), suppression des stopwords, racinisation de chaque mot individuellement.

In [17]:
# Fonction clean générale
def clean_txt(txt):
    ### remove html stuff
    txt = BeautifulSoup(txt,"html.parser",from_encoding='utf-8').get_text()
    ### lower case
    txt = txt.lower()
    ### special escaping character '...'
    txt = txt.replace(u'\u2026','.')
    txt = txt.replace(u'\u00a0',' ')
    ### remove accent btw
    txt = unicodedata.normalize('NFD', txt).encode('ascii', 'ignore').decode("utf-8")
    ###txt = unidecode(txt)
    ### remove non alphanumeric char
    txt = re.sub('[^a-z_]', ' ', txt)
    ### remove french stop words
    tokens = [w for w in txt.split() if (len(w)>2) and (w not in stopwords)]
    ### french stemming
    tokens_stem = [stemmer.stem(token) for token in tokens]
    ### tokens = stemmer.stemWords(tokens)
    return ' '.join(tokens), " ".join(tokens_stem)

def clean_marque(txt):
    txt = re.sub('[^a-zA-Z0-9]', '_', txt).lower()
    return txt

### Nettoyage des DataFrames
Applique le nettoyage sur toutes les lignes de la DataFrame

In [18]:
# fonction de nettoyage du fichier(stemming et liste de mots à supprimer)
def clean_df(input_data, column_names= ['Description', 'Libelle', 'Marque']):
    #Test if columns entry match columns names of input data
    column_names_diff= set(column_names).difference(set(input_data.columns))
    if column_names_diff:
        warnings.warn("Column(s) '"+", ".join(list(column_names_diff)) +"' do(es) not match columns of input data", Warning)
    
    nb_line = input_data.shape[0]
    print("Start Clean %d lines" %nb_line)
    
    # Cleaning start for each columns
    time_start = time.time()
    clean_list=[]
    clean_stem_list=[]
    for column_name in column_names:
        column = input_data[column_name].values
        if column_name == "Marque":
            array_clean = np.array(list(map(clean_marque,column)))
        else:
            A = np.array(list(map(clean_txt,column)))
            array_clean = A[:,0]
            array_clean_stem = A[:,1]
        clean_list.append(array_clean)
        clean_stem_list.append(array_clean_stem)
    time_end = time.time()
    print("Cleaning time: %d secondes"%(time_end-time_start))
    
    #Convert list to DataFrame
    array_clean = np.array(clean_list).T
    data_clean = pd.DataFrame(array_clean, columns = column_names)
    
    array_clean_stem = np.array(clean_stem_list).T
    data_clean_stem = pd.DataFrame(array_clean_stem, columns = column_names)
    return data_clean, data_clean_stem

In [21]:
# Take approximately 2 minutes fors 100.000 rows
data_valid_clean, data_valid_clean_stem = clean_df(data_valid)

Start Clean 100000 lines


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/bs4/__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


Cleaning time: 91 secondes


In [22]:
data_valid_clean.values[0], data_valid_clean_stem.values[0]

(array([ 'batterie pavilion mah ion batterie pavilion mah ioncaracteri voir presentation',
        'batterie pavilion mah', 'hp'], dtype=object),
 array(['batter pavilion mah ion batter pavilion mah ioncaracter voir present',
        'batter pavilion mah', 'batter pavilion mah'], dtype=object))

In [23]:
data_train_clean, data_train_clean_stem = clean_df(data_train)

Start Clean 900000 lines


/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/bs4/__init__.py:146: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/bguillouet/anaconda/envs/TPInsa/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "http://www.sportsdepot.fr/SPORTS_DEPOT/pics/part/7" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/Users/bguil

Cleaning time: 73220 secondes


Affiche les 5 premières lignes de la DataFrame d'apprentissage après nettoyage.

In [25]:
data_train_clean.head(5)

,Description,Libelle,Marque
0,fermeture boutonnee maille fine toucher doux b...,gilet col cachemire,aucune
1,coque souple rose htc one mini motif love bang...,coque souple rose htc one mini motif lov,muzzano
2,coque souple grise wiko stairway motif drapeau...,coque souple grise wiko stairway motif dra,muzzano
3,samsung galaxy batterie haute densite origine ...,samsung galaxy batterie haute den,samsung
4,batterie sony vaio vgn ion mah noir compatible...,batterie sony vaio vgn,aucune


In [26]:
data_train_clean_stem.head(5)

,Description,Libelle,Marque
0,fermetur boutonne maill fin touch doux bord co...,gilet col cachemir,gilet col cachemir
1,coqu soupl ros htc one min motif lov bangkok f...,coqu soupl ros htc one min motif lov,coqu soupl ros htc one min motif lov
2,coqu soupl gris wiko stairway motif drapeau is...,coqu soupl gris wiko stairway motif dra,coqu soupl gris wiko stairway motif dra
3,samsung galaxy batter haut densit origin samsu...,samsung galaxy batter haut den,samsung galaxy batter haut den
4,batter sony vaio vgn ion mah noir compatibl ba...,batter sony vaio vgn,batter sony vaio vgn


Count Vocabulary size

In [35]:
concatenate_text = " ".join(data_train_clean["Description"].values)
list_of_word = concatenate_text.split(" ")
N = len(set(list_of_word))
print(N)
print(list_of_word[:10])

252126
['fermeture', 'boutonnee', 'maille', 'fine', 'toucher', 'doux', 'bords', 'coteles', 'cachemire', 'soie']


In [36]:
concatenate_text = " ".join(data_train_clean_stem["Description"].values)
list_of_word_stem = concatenate_text.split(" ")
N = len(set(list_of_word_stem))
print(N)
print(list_of_word_stem[:10])

202289
['fermetur', 'boutonne', 'maill', 'fin', 'touch', 'doux', 'bord', 'cotel', 'cachemir', 'soi']


Save valide and train clean DF

In [27]:
data_valid_clean.to_csv(DATA_DIR + "Categorie_reduit_valid_clean.csv", index=False)
data_train_clean.to_csv(DATA_DIR + "Categorie_reduit_train_clean.csv", index=False)

data_valid_clean_stem.to_csv(DATA_DIR + "Categorie_reduit_valid_clean_stem.csv", index=False)
data_train_clean_stem.to_csv(DATA_DIR + "Categorie_reduit_train_clean_stem.csv", index=False)

In [39]:
CC = collections.Counter(list_of_word)
CC_stem = collections.Counter(list_of_word_stem)

In [41]:
CC["coqu"],CC["coque"]

(247, 246712)

In [43]:
CC_stem["coqu"], CC_stem["coque"]

(248137, 11)

'homm'